In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
from sklearn.neighbors import BallTree
import warnings
warnings.filterwarnings('ignore')

# Настройка OSMnx
ox.settings.log_console = True
ox.settings.use_cache = True

# %%
# Загрузим основной датасет
df = pd.read_csv('test.tsv', sep='\t')

# Вытащим координаты
df['lon'] = df['coordinates'].apply(lambda x: float(x.strip('[]').split(',')[0]))
df['lat'] = df['coordinates'].apply(lambda x: float(x.strip('[]').split(',')[1]))

print(f"Loaded df with shape: {df.shape}")

# %%
# ======================
# 1. РЕАЛЬНЫЕ ДАННЫЕ МЕТРО ИЗ OSM
# ======================
print("Downloading real metro stations from OSM...")

try:
    # Используем работающий метод features_from_place
    metro_gdf = ox.features_from_place('Moscow, Russia', tags={'railway': 'station'})
    
    # Фильтруем только станции метро
    if 'station' in metro_gdf.columns:
        metro_gdf = metro_gdf[metro_gdf['station'] == 'subway']
    elif 'public_transport' in metro_gdf.columns:
        metro_gdf = metro_gdf[metro_gdf['public_transport'] == 'station']
    
    metro_gdf = metro_gdf.to_crs(epsg=4326)
    print(f"Found {len(metro_gdf)} real subway stations from OSM")
    
    # Сохраним для проверки
    print("Sample metro stations:")
    for i, (idx, row) in enumerate(metro_gdf.head().iterrows()):
        if hasattr(row.geometry, 'centroid'):
            centroid = row.geometry.centroid
            print(f"  {i+1}. {row.get('name', 'Unknown')} - ({centroid.y:.4f}, {centroid.x:.4f})")
    
except Exception as e:
    print(f"Error fetching real metro data: {e}")
    metro_gdf = None

# %%
# ======================
# 2. РЕАЛЬНЫЕ КЛЮЧЕВЫЕ ТОЧКИ МОСКВЫ
# ======================
print("Adding real Moscow key locations...")

# РЕАЛЬНЫЕ координаты ключевых объектов Москвы (из открытых источников)
real_key_locations = {
    # Правительственные здания
    'kremlin': (55.7520, 37.6175),  # Московский Кремль
    'white_house': (55.7570, 37.6170),  # Дом Правительства РФ
    'moscow_city_hall': (55.7558, 37.6177),  # Мэрия Москвы
    
    # Транспортные узлы
    'komsomolskaya_metro': (55.7735, 37.6563),  # Комсомольская (3 вокзала)
    'kievskaya_metro': (55.7437, 37.5672),  # Киевская (Киевский вокзал)
    'belorusskaya_metro': (55.7764, 37.5842),  # Белорусская (Белорусский вокзал)
    'kurskaya_metro': (55.7586, 37.6611),  # Курская (Курский вокзал)
    'paveletskaya_metro': (55.7314, 37.6361),  # Павелецкая (Павелецкий вокзал)
    
    # Крупные торговые центры
    'evropeiskiy_mall': (55.7415, 37.5320),  # ТЦ Европейский
    'afimall_city': (55.7496, 37.5394),  # Афимолл Сити
    'gum': (55.7547, 37.6215),  # ГУМ
    'tsum': (55.7600, 37.6247),  # ЦУМ
    
    # Культурные объекты
    'bolshoi_theatre': (55.7601, 37.6186),  # Большой театр
    'tretyakov_gallery': (55.7415, 37.6208),  # Третьяковская галерея
    'pushkin_museum': (55.7473, 37.6053),  # Музей изобразительных искусств
    'vdnh': (55.8294, 37.6316),  # ВДНХ
    
    # Парки и общественные пространства
    'gorky_park': (55.7280, 37.5985),  # Парк Горького
    'zaryadye_park': (55.7514, 37.6270),  # Парк Зарядье
    'vorobyovy_gory': (55.7100, 37.5594),  # Воробьевы горы
    
    # Образовательные учреждения
    'msu': (55.7039, 37.5289),  # МГУ
    'mpei': (55.7319, 37.7651),  # МЭИ
    'baumanka': (55.7657, 37.6834),  # МГТУ Бауманка
    
    # Деловые центры
    'moscow_city_business': (55.7496, 37.5394),  # Москва-Сити
    'krasnopresnenskaya_embankment': (55.7593, 37.5413),  # Краснопресненская наб.
    
    # Аэропорты (реальные координаты терминалов)
    'sheremetyevo_terminal_b': (55.9726, 37.4146),  # Шереметьево терминал B
    'sheremetyevo_terminal_c': (55.9742, 37.4150),  # Шереметьево терминал C
    'sheremetyevo_terminal_d': (55.4089, 37.9063),  # Шереметьево терминал D
    'domodedovo_terminal': (55.4143, 37.9005),  # Домодедово
    'vnukovo_terminal_a': (55.5969, 37.2850),  # Внуково терминал A
}

print(f"Using {len(real_key_locations)} real key locations in Moscow")

# %%
# ======================
# 3. РАССТОЯНИЯ ДО РЕАЛЬНЫХ ОБЪЕКТОВ
# ======================
print("Calculating distances to real locations...")

coords = df[['lat', 'lon']].values

# Расстояния до ключевых объектов
for name, (loc_lat, loc_lon) in real_key_locations.items():
    key_coords = np.array([[loc_lat, loc_lon]])
    tree = BallTree(np.deg2rad(key_coords), metric='haversine')
    distances, _ = tree.query(np.deg2rad(coords), k=1)
    df[f'dist_to_{name}_km'] = distances[:, 0] * 6371

print("Added real location distance features")

# %%
# ======================
# 4. РАССТОЯНИЯ ДО МЕТРО (ЕСЛИ ЕСТЬ РЕАЛЬНЫЕ ДАННЫЕ)
# ======================
if metro_gdf is not None and not metro_gdf.empty:
    print("Calculating distances to real metro stations...")
    
    metro_centroids = metro_gdf.geometry.centroid
    metro_coords = np.column_stack([metro_centroids.y, metro_centroids.x])
    
    tree = BallTree(np.deg2rad(metro_coords), metric='haversine')
    
    # Ближайшая станция
    distances, indices = tree.query(np.deg2rad(coords), k=1)
    df['dist_to_nearest_metro_km'] = distances[:, 0] * 6371
    
    # 3 ближайшие станции
    distances, indices = tree.query(np.deg2rad(coords), k=3)
    for i in range(3):
        df[f'dist_to_metro_{i+1}_km'] = distances[:, i] * 6371
    
    # Плотность станций метро в радиусе 2 км
    counts_2km = tree.query_radius(np.deg2rad(coords), r=2.0/6371, count_only=True)
    df['metro_stations_2km'] = counts_2km
    
    print("Added real metro distance features")
else:
    print("Using only key locations (no metro data available)")

# %%
# ======================
# 5. ГЕОГРАФИЧЕСКИЕ ХАРАКТЕРИСТИКИ
# ======================
print("Adding geographic characteristics...")

# Центр Москвы (исторический центр - Кремль)
center_coords = np.array([[55.7520, 37.6175]])

# Расстояние до центра
tree_center = BallTree(np.deg2rad(center_coords), metric='haversine')
center_distances, _ = tree_center.query(np.deg2rad(coords), k=1)
df['dist_to_center_km'] = center_distances[:, 0] * 6371

# Кольца Москвы (приблизительные границы)
sadovoe_ring_coords = np.array([[55.7600, 37.6200]])  # Садовое кольцо
ttk_coords = np.array([[55.7800, 37.6000]])  # Третье транспортное кольцо
mkad_coords = np.array([[55.8500, 37.5000]])  # МКАД

for ring_name, ring_coords in [('sadovoe_ring', sadovoe_ring_coords), 
                              ('ttk', ttk_coords), 
                              ('mkad', mkad_coords)]:
    tree_ring = BallTree(np.deg2rad(ring_coords), metric='haversine')
    ring_distances, _ = tree_ring.query(np.deg2rad(coords), k=1)
    df[f'dist_to_{ring_name}_km'] = ring_distances[:, 0] * 6371

# Плотность всех объектов датасета в радиусе 1 км
tree_all = BallTree(np.deg2rad(coords), metric='haversine')
counts_1km = tree_all.query_radius(np.deg2rad(coords), r=1.0/6371, count_only=True)
df['object_density_1km'] = counts_1km

print("Added geographic characteristics")

# %%
# ======================
# 6. КЛАСТЕРИЗАЦИЯ ПО РЕАЛЬНЫМ ГЕОГРАФИЧЕСКИМ ЗОНАМ
# ======================
print("Adding geographic clustering...")

from sklearn.cluster import KMeans

# Создаем кластеры на основе реальных координат
coords_for_clustering = df[['lat', 'lon']].values
kmeans = KMeans(n_clusters=15, random_state=42, n_init=10)
df['geo_cluster'] = kmeans.fit_predict(coords_for_clustering)

# One-hot encoding для кластеров
cluster_dummies = pd.get_dummies(df['geo_cluster'], prefix='geo_cluster')
df = pd.concat([df, cluster_dummies], axis=1)

print("Added geographic clustering")

# %%
# ======================
# 7. АГРЕГАЦИЯ СУЩЕСТВУЮЩИХ ФИЧ ПО КЛАСТЕРАМ
# ======================
print("Aggregating existing features by clusters...")

# Выбираем только числовые колонки для агрегации
numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
# Исключаем координаты, целевые и временные
exclude_cols = ['lat', 'lon', 'geo_cluster']
numeric_columns = [col for col in numeric_columns if col not in exclude_cols]

# Берем только первые 15 колонок чтобы не перегружать
selected_columns = numeric_columns[:15]

# Средние значения по кластерам для выбранных колонок
cluster_means = df.groupby('geo_cluster')[selected_columns].mean()

for col in selected_columns:
    cluster_means_col = cluster_means[col].to_dict()
    df[f'cluster_mean_{col}'] = df['geo_cluster'].map(cluster_means_col)

print("Added cluster aggregation features")

# %%
# ======================
# 8. ФИНАЛЬНЫЕ ВЫЧИСЛЕНИЯ НА ОСНОВЕ РЕАЛЬНЫХ ДАННЫХ
# ======================
print("Final computations based on real data...")

# Индекс транспортной доступности (чем ближе к метро и центру - тем лучше)
if 'dist_to_nearest_metro_km' in df.columns:
    df['transport_accessibility'] = 1 / (1 + df['dist_to_nearest_metro_km'] * 0.5 + df['dist_to_center_km'] * 0.1)
else:
    df['transport_accessibility'] = 1 / (1 + df['dist_to_center_km'] * 0.2)

# Коммерческая привлекательность (основанная на реальных расстояниях)
commercial_hubs = ['kremlin', 'white_house', 'moscow_city_business', 'evropeiskiy_mall', 'gum']
commercial_distances = sum([df[f'dist_to_{hub}_km'] for hub in commercial_hubs]) / len(commercial_hubs)
df['commercial_potential'] = 1 / (1 + commercial_distances * 0.1)

print("Added final computed features")

# %%
# ======================
# 9. СОХРАНЕНИЕ И ИНФОРМАЦИЯ
# ======================
output_file = 'test_with_real_geo_features.csv'
df.to_csv(output_file, index=False)

print(f"Saved dataset with {df.shape[1]} features to {output_file}")

# Список добавленных гео-фичей
new_geo_features = [col for col in df.columns if any(x in col for x in [
    'dist_to_', 'metro_', 'cluster_', 'density', 'accessibility', 'potential'
])]

print(f"\nAdded {len(new_geo_features)} REAL geo features based on actual Moscow geography:")
for feature in new_geo_features:
    print(f"  - {feature}")

print(f"\nTotal features in dataset: {df.shape[1]}")
print(f"Total rows: {df.shape[0]}")

# Информация об источниках данных
print("\n" + "="*50)
print("DATA SOURCES:")
print("1. Metro stations: OpenStreetMap via OSMnx")
print("2. Key locations: Real coordinates from public maps")
print("3. Geographic features: Calculated from actual coordinates")
print("4. All distances: Haversine formula on real coordinates")
print("="*50)